In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from itertools import product
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
batch_size = 32
num_epochs = 20  # Will be dynamically adjusted during hyperparameter tuning
learning_rates = [1e-2, 1e-3]
step_sizes = [30, 50]

# Data loading and preprocessing
data_dir = '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/Training'  # Update with actual data directory path
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# Model definition
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Update to 5 for multi-class classification
model = model.to(device)


# Loss function
criterion = nn.CrossEntropyLoss()

# Hyperparameter tuning setup
hyperparameters = list(product(learning_rates, step_sizes, [num_epochs]))
best_accuracy = 0.0
best_params = None


In [6]:
class ResNet152(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet152, self).__init__()
        self.model = models.resnet152(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.model(x)

model = ResNet152(num_classes=5).to(device)

In [7]:

for lr, step_size, num_epochs in hyperparameters:
    # Redefine optimizer and scheduler with new hyperparameters
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct / total

        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_loss = running_loss / len(val_loader.dataset)
        val_accuracy = correct / total

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = (lr, step_size, num_epochs)
            torch.save(model.state_dict(), 'best_model.pth')
        
        scheduler.step()

print(f'Best accuracy was {best_accuracy:.2f} with lr={best_params[0]}, step size={best_params[1]}, and no. of epochs={best_params[2]}')


Epoch 1/20, Train Loss: 1.4816, Train Accuracy: 0.4409, Val Loss: 6.4693, Val Accuracy: 0.2758
Epoch 2/20, Train Loss: 1.3722, Train Accuracy: 0.5015, Val Loss: 2.1407, Val Accuracy: 0.4273
Epoch 3/20, Train Loss: 1.0698, Train Accuracy: 0.5818, Val Loss: 0.9438, Val Accuracy: 0.5970


KeyboardInterrupt: 